## Задание №5. Разработка программы построения и оценки линейной многофакторной модели

Разработать программу, которая будет считывать временные ряды отклика y и факторов Х из текстового файла и выполнять следующие операции

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from math import sqrt
import sys

## Оценка параметров линейной многофакторной модели (ЛМФМ).

In [2]:
def estimate_parameters(X, y):
    """
    Оценка параметров модели с помощью МНК.
    """
    X = sm.add_constant(X)  # Добавление свободного члена
    model = sm.OLS(y, X).fit()
    return model

### Линейная многофакторная модель описывает зависимость зависимой переменной y от нескольких независимых переменных (факторов) X1, X2, ..., Xn в виде:

$y = b0 + b1*X1 + b2*X2 + ... + bn*Xn + e$

где:

$b0$ — свободный член (интерцепт),
$b1, b2, ..., bn$ — коэффициенты модели для соответствующих факторов,
$e$ — случайная ошибка.
Как ваша программа оценивает параметры ЛМФМ
В вашей программе оценка параметров модели производится с помощью функции estimate_parameters. Рассмотрим этот процесс поэтапно.


1. Добавление свободного члена (интерцепта)
```python
sm.add_constant(X)
```
Эта функция из библиотеки statsmodels добавляет столбец единиц к матрице факторов X. Это необходимо для оценки интерцепта (b0) модели. Без этого шага модель будет принудительно проходить через начало координат, что не всегда соответствует реальности.


2. Оценка параметров с помощью метода наименьших квадратов (МНК)

```python
model = sm.OLS(y, X).fit()
```

`sm.OLS(y, X)`: Создаёт объект модели линейной регрессии, где y — зависимая переменная, а X — матрица независимых переменных (с добавленным столбцом единиц для интерцепта).

`.fit()`: Выполняет оценку параметров модели с помощью метода наименьших квадратов. Этот метод находит такие значения коэффициентов b0, b1, ..., bn, которые минимизируют сумму квадратов разностей между наблюдаемыми значениями y_i и предсказанными значениями yhat_i.



### Статистические показатели: R^2, скорректированный R^2, F-статистика, p-значения для каждого коэффициента, стандартные ошибки и доверительные интервалы.
Пример вывода model.summary()

```
                             OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                     3751.
Date:                Wed, 16 Oct 2024   Prob (F-statistic):          7.06e-191
Time:                        21:59:12   Log-Likelihood:                -412.04
No. Observations:                 200   AIC:                             836.1
Df Residuals:                     194   BIC:                             855.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2048      0.551     -0.372      0.710      -1.291       0.881
X1             2.4332      0.046     53.336      0.000       2.343       2.523
X2             1.5882      0.047     33.675      0.000       1.495       1.681
X3             2.9998      0.046     65.509      0.000       2.909       3.090
X4             4.5110      0.050     89.598      0.000       4.412       4.610
X5             1.9700      0.048     40.894      0.000       1.875       2.065
==============================================================================
Omnibus:                        7.061   Durbin-Watson:                   1.941
Prob(Omnibus):                  0.029   Jarque-Bera (JB):                6.774
Skew:                          -0.410   Prob(JB):                       0.0338
Kurtosis:                       3.373   Cond. No.                         47.2
==============================================================================
```
$coef$: Оценённые значения коэффициентов (b).

$const$: Значение интерцепта (b0).
$X1, X2, X3$: Коэффициенты для соответствующих факторов.
$std err$: Стандартная ошибка оценки коэффициента.

$t$: t-статистика для проверки гипотезы о значимости коэффициента.

$P>|t|$: p-значение для проверки гипотезы.

$[0.025 0.975]$: 95% доверительный интервал для коэффициента.

### Какие параметры оцениваются?
В данной модели оцениваются следующие параметры:

- Свободный член (b0):

Представляет собой базовое значение зависимой переменной y, когда все независимые переменные X1, X2, ..., Xn равны нулю.
Оценён автоматически благодаря добавлению столбца единиц с помощью sm.add_constant(X).


- Коэффициенты факторов (b1, b2, ..., bn):

Отражают изменение зависимой переменной y при изменении соответствующего фактора Xi на единицу, при условии, что остальные факторы остаются неизменными.
Эти коэффициенты показывают влияние каждого фактора на y.



## Отбор значимых факторов (решение об отсеве факторов, возврате к повторной процедуре отсева и т.д. принимает пользователь)


### 1) Отбор значимых факторов по статистической значимости (критерий выбирается разработчиком, но уровень значимости - пользователем);

In [3]:
def select_significant_factors(model, significance_level):
    """
    Отбор значимых факторов по статистической значимости.
    Возвращает список значимых факторов.
    """
    p_values = model.pvalues
    significant = p_values[p_values < significance_level].index.tolist()
    if "const" in significant:
        significant.remove("const")
    return significant


Описание:

`model.pvalues`: Получает p-значения для всех коэффициентов модели, включая свободный член (const).

`Отбор`: Факторы с p-значением меньше заданного уровня значимости (significance_level) считаются значимыми.

`Исключение интерцепта`: Свободный член (const) удаляется из списка значимых факторов, так как он не является фактором.

### 2) Отбор по коэффициенту корреляции - связь между факторами (между собой)

In [4]:
def check_multicollinearity(X, threshold=0.8):
    """
    Проверка мультиколлинеарности между факторами.
    Возвращает пары факторов с коэффициентом корреляции выше порога.
    """
    corr_matrix = X.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    high_corr = [
        (column, row)
        for column in upper.columns
        for row in upper.index
        if upper.loc[row, column] > threshold
    ]
    return high_corr


Описание:

`X.corr().abs()`: Вычисляет абсолютные значения коэффициентов корреляции между всеми парами факторов.

`upper`: Извлекает верхнюю треугольную часть матрицы корреляции, чтобы избежать повторений и самокорреляции.

`Отбор`: Создаёт список пар факторов, где коэффициент корреляции превышает заданный порог (threshold).

### 3) Отбор по коэффициенту корреляции - связь между факторами и откликом

In [5]:
def check_correlation_with_response(X, y, threshold=0.3):
    """
    Проверка корреляции между факторами и откликом.
    
    Эта функция вычисляет коэффициент корреляции между каждым фактором в X и зависимой переменной y.
    Она возвращает список факторов, коэффициент корреляции которых по абсолютной величине превышает заданный порог.
    
    Параметры:
    - X (pd.DataFrame): DataFrame, содержащий независимые переменные (факторы).
    - y (pd.Series): Series, содержащий зависимую переменную (отклик).
    - threshold (float): Пороговое значение для коэффициента корреляции. Факторы с |corr| > threshold будут отобраны.
    
    Возвращает:
    - significant_corr (list): Список названий факторов, удовлетворяющих условию корреляции.
    """
    # Вычисление коэффициентов корреляции между каждым фактором и откликом y
    correlations = X.apply(lambda col: col.corr(y))
    
    # Отбор факторов, абсолютное значение корреляции которых превышает порог threshold
    significant_corr = correlations[correlations.abs() > threshold].index.tolist()
    
    return significant_corr


### Критерии для Исключения Факторов
- Статистическая Значимость (p-значение):

Что: Показывает, насколько фактор важен для модели.
Как: Если p-значение фактора выше выбранного уровня значимости (например, 0.05), фактор считается статистически незначимым и может быть исключен.

- Мультиколлинеарность (Корреляция между Факторами):

Что: Означает сильную взаимосвязь между двумя или более факторами.
Как: Используйте корреляционную матрицу или фактор инфляции дисперсии (VIF). Если корреляция между факторами превышает порог (например, 0.8), один из них следует удалить, чтобы избежать искажений в модели.

- Корреляция с Откликом:

Что: Показывает, насколько сильно фактор связан с зависимой переменной.
Как: Факторы с низкой корреляцией с откликом (например, ниже 0.3) могут иметь малое влияние на модель и быть кандидатами на исключение.

## Оценка адекватности модели


### Оценка коэффициента детерминации на статистическую значимость (F - статистика) и сообщение: адекватна модель или нет;
Коэффициент детерминации $R^2$ показывает, какая доля вариации зависимой переменной объясняется моделью.

F-статистика используется для проверки общей значимости модели. Если p-значение F-статистики меньше уровня значимости, модель считается адекватной.


### Вычисление ошибки $(E = 1/n * sum ((yi-yit)/yit)$ или RMSE

RMSE (Root Mean Squared Error) измеряет среднюю величину ошибок предсказания модели. Чем ниже значение RMSE, тем точнее модель.

In [6]:
def evaluate_model_adadequacy(model, X, y):
    """
    Оценка адекватности модели.
    Возвращает R^2, F-статистику, p-значение F-статистики и RMSE.
    """
    r_squared = model.rsquared  # Коэффициент детерминации R^2
    f_stat = model.fvalue        # F-статистика
    f_pvalue = model.f_pvalue    # p-значение для F-статистики
    predictions = model.predict(X)  # Предсказанные значения y
    rmse = sqrt(mean_squared_error(y, predictions))  # RMSE
    return r_squared, f_stat, f_pvalue, rmse


### Вычисление и вывод пользователю специальных показателей.

In [7]:
def special_metrics(y, predictions):
    """
    Вычисление специальных показателей.
    Здесь вычисляется относительная ошибка.
    """
    relative_error = np.mean(np.abs((y - predictions) / y))  # Относительная ошибка
    return relative_error


Функция special_metrics рассчитывает относительную ошибку как среднее значение абсолютных относительных отклонений предсказанных значений от фактических. Это значение также выводится пользователю для дополнительной оценки точности модели.

## Выполнения предсказания с помощью построенной модели на основе ввода значений факторов 

In [8]:
def predict_new(model, new_data_path):
    """
    Предсказание новых значений отклика на основе введённых факторов.
    """
    try:
        # Чтение новых данных из файла CSV
        new_data = pd.read_csv(new_data_path, sep=None, engine="python")
        
        # Добавление столбца единиц для свободного члена (интерцепта)
        X_new = sm.add_constant(new_data)
        
        # Выполнение предсказания с помощью модели
        predictions = model.predict(X_new)
        
        # Возврат предсказанных значений
        return predictions
    except Exception as e:
        # Вывод ошибки и завершение программы в случае исключения
        print(f"Ошибка при чтении файла с новыми данными: {e}")
        sys.exit(1)


## Полный код программы 

In [9]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from math import sqrt
import sys



def read_data(file_path):
    """
    Чтение данных из текстового файла.
    Предполагается, что первый столбец — отклик y,
    а остальные — факторы X1, X2, ..., Xn.
    """
    try:
        data = pd.read_csv(
            file_path, sep=None, engine="python"
        )  # Автоматическое определение разделителя
        return data
    except Exception as e:
        print(f"Ошибка при чтении файла: {e}")
        sys.exit(1)


def estimate_parameters(X, y):
    """
    Оценка параметров модели с помощью МНК.
    """
    X = sm.add_constant(X)  # Добавление свободного члена
    model = sm.OLS(y, X).fit()
    return model


def select_significant_factors(model, significance_level):
    """
    Отбор значимых факторов по статистической значимости.
    Возвращает список значимых факторов.
    """
    p_values = model.pvalues
    significant = p_values[p_values < significance_level].index.tolist()
    if "const" in significant:
        significant.remove("const")
    return significant


def check_multicollinearity(X, threshold=0.8):
    """
    Проверка мультиколлинеарности между факторами.
    Возвращает пары факторов с коэффициентом корреляции выше порога.
    """
    corr_matrix = X.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    high_corr = [
        (column, row)
        for column in upper.columns
        for row in upper.index
        if upper.loc[row, column] > threshold
    ]
    return high_corr


def evaluate_model_adadequacy(model, X, y):
    """
    Оценка адекватности модели.
    Возвращает информацию о R^2, F-статистике и RMSE.
    """
    r_squared = model.rsquared
    f_stat = model.fvalue
    f_pvalue = model.f_pvalue
    predictions = model.predict(X)
    rmse = sqrt(mean_squared_error(y, predictions))
    return r_squared, f_stat, f_pvalue, rmse


def special_metrics(y, predictions):
    """
    Вычисление специальных показателей.
    Здесь можно добавить любые дополнительные метрики.
    """
    relative_error = np.mean(np.abs((y - predictions) / y))
    return relative_error


def predict_new(model, new_data_path):
    """
    Предсказание новых значений отклика на основе введённых факторов.
    """
    try:
        new_data = pd.read_csv(new_data_path, sep=None, engine="python")
        X_new = sm.add_constant(new_data)
        predictions = model.predict(X_new)
        return predictions
    except Exception as e:
        print(f"Ошибка при чтении файла с новыми данными: {e}")
        sys.exit(1)


def main():

    # Пути к файлам (можно заменить на ввод пользователя)
    data_file = "data.csv"  # Входные данные: первый столбец y, остальные X
    new_data_file = "new_data.csv"  # Новые факторы для предсказания
    output_file = "output.txt"  # Файл для сохранения результатов

    # Чтение данных
    data = read_data(data_file)
    y = data.iloc[:, 0]
    X = data.iloc[:, 1:]

    # Ввод уровня значимости
    try:
        significance_level = float(
            input("Введите уровень значимости (например, 0.05): ")
        )
    except ValueError:
        print("Некорректный ввод. Используется уровень значимости 0.05.")
        significance_level = 0.05

    # Оценка параметров модели
    model = estimate_parameters(X, y)
    print("Параметры модели:\n", model.summary())

    # Отбор значимых факторов
    significant_factors = select_significant_factors(model, significance_level)
    print(f"Значимые факторы (p < {significance_level}): {significant_factors}")

    # Проверка мультиколлинеарности
    if significant_factors:
        high_corr = check_multicollinearity(X[significant_factors])
        if high_corr:
            print("Факторы с высокой корреляцией между собой:")
            for pair in high_corr:
                print(f"{pair[0]} и {pair[1]}")
        else:
            print("Мультиколлинеарности между факторами не обнаружено.")
    else:
        print("Нет значимых факторов для проверки мультиколлинеарности.")

    # Оценка адекватности модели
    X_with_const = sm.add_constant(X)
    r2, f_stat, f_pval, rmse = evaluate_model_adadequacy(model, X_with_const, y)
    print(f"Коэффициент детерминации R^2: {r2}")
    print(f"F-статистика: {f_stat}, p-значение: {f_pval}")
    print(f"RMSE: {rmse}")

    if f_pval < significance_level:
        print("Модель адекватна.")
    else:
        print("Модель неадекватна.")

    # Вычисление специальных показателей
    predictions = model.predict(X_with_const)
    rel_error = special_metrics(y, predictions)
    print(f"Относительная ошибка: {rel_error}")

    # Предсказание на новых данных
    user_choice = (
        input("Хотите выполнить предсказание на новых данных? (да/нет): ")
        .strip()
        .lower()
    )
    if user_choice == "да":
        predictions_new = predict_new(model, new_data_file)
        print("Предсказанные значения для новых данных:")
        print(predictions_new)
    else:
        print("Предсказание не выполнено.")

    # Сохранение результатов в файл
    with open(output_file, "w", encoding="utf-8") as f:
        f.write("Параметры модели:\n")
        f.write(model.summary().as_text())
        f.write("\nЗначимые факторы:\n")
        f.write(
            ", ".join(significant_factors)
            if significant_factors
            else "Нет значимых факторов."
        )
        f.write("\nМультиколлинеарность:\n")
        if significant_factors and high_corr:
            for pair in high_corr:
                f.write(f"{pair[0]} и {pair[1]}\n")
        elif significant_factors:
            f.write("Мультиколлинеарность не обнаружена.\n")
        else:
            f.write("Нет значимых факторов для проверки мультиколлинеарности.\n")
        f.write(f"\nКоэффициент детерминации R^2: {r2}\n")
        f.write(f"F-статистика: {f_stat}, p-значение: {f_pval}\n")
        f.write(f"RMSE: {rmse}\n")
        f.write(f"Относительная ошибка: {rel_error}\n")
        if user_choice == "да":
            f.write("\nПредсказанные значения для новых данных:\n")
            f.write(predictions_new.to_string())
    print(f"Результаты сохранены в файл '{output_file}'.")


if __name__ == "__main__":
    main()


Параметры модели:
                             OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                     3751.
Date:                Thu, 17 Oct 2024   Prob (F-statistic):          7.06e-191
Time:                        00:18:01   Log-Likelihood:                -412.04
No. Observations:                 200   AIC:                             836.1
Df Residuals:                     194   BIC:                             855.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2048      0.551 

### Результат работы программы
```sh
Параметры модели:
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                     3751.
Date:                Thu, 17 Oct 2024   Prob (F-statistic):          7.06e-191
Time:                        00:13:47   Log-Likelihood:                -412.04
No. Observations:                 200   AIC:                             836.1
Df Residuals:                     194   BIC:                             855.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2048      0.551     -0.372      0.710      -1.291       0.881
X1             2.4332      0.046     53.336      0.000       2.343       2.523
X2             1.5882      0.047     33.675      0.000       1.495       1.681
X3             2.9998      0.046     65.509      0.000       2.909       3.090
X4             4.5110      0.050     89.598      0.000       4.412       4.610
X5             1.9700      0.048     40.894      0.000       1.875       2.065
==============================================================================
Omnibus:                        7.061   Durbin-Watson:                   1.941
Prob(Omnibus):                  0.029   Jarque-Bera (JB):                6.774
Skew:                          -0.410   Prob(JB):                       0.0338
Kurtosis:                       3.373   Cond. No.                         47.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
Значимые факторы:
X1, X2, X3, X4, X5
Мультиколлинеарность:
Мультиколлинеарность не обнаружена.

Коэффициент детерминации R^2: 0.9897620975121254
F-статистика: 3751.0387922675823, p-значение: 7.058135919128105e-191
RMSE: 1.898844671791363
Относительная ошибка: 0.024724024385834733

Предсказанные значения для новых данных:
0      60.223969
1      66.352848
2      60.435731
3      46.202399
4      78.349259
5      51.924627
6      36.952918
7      89.348472
8      77.532742
9      51.479849
10     67.895967
11     84.836578
12    101.684011
13     22.691974
14     72.738586
15     64.708120
16     57.041787
17     82.525183
18     85.726109
19     32.860691
```

### Для генерации исходных данных использовался следующий скрипт

In [10]:
import pandas as pd
import numpy as np


import argparse


def generate_data(
    n_samples=100,
    n_factors=3,
    coefficients=None,
    noise_level=1.0,
    output_file="data.csv",
    new_data_file="new_data.csv",
    n_new_samples=10,
    random_state=None,
):
    """
    Генерация синтетических данных для линейной многофакторной модели.

    Параметры:
    - n_samples (int): Количество наблюдений.
    - n_factors (int): Количество факторов.
    - coefficients (list или array): Коэффициенты для факторов. Длина должна быть равна n_factors.
                                       Если None, коэффициенты генерируются случайно.
    - noise_level (float): Стандартное отклонение шума.
    - output_file (str): Имя выходного файла для исходных данных.
    - new_data_file (str): Имя выходного файла для новых данных.
    - n_new_samples (int): Количество новых наблюдений для предсказания.
    - random_state (int или None): Сид для генератора случайных чисел для воспроизводимости.

    Возвращает:
    - None
    """
    if random_state is not None:
        np.random.seed(random_state)

    # Генерация коэффициентов, если они не заданы
    if coefficients is None:
        coefficients = np.random.uniform(1, 10, size=n_factors)
    else:
        coefficients = np.array(coefficients)
        if len(coefficients) != n_factors:
            raise ValueError("Длина списка коэффициентов должна совпадать с n_factors.")

    # Генерация факторов X
    X = np.random.uniform(0, 10, size=(n_samples, n_factors))
    X_df = pd.DataFrame(X, columns=[f"X{i+1}" for i in range(n_factors)])

    # Генерация отклика y
    y = X @ coefficients + np.random.normal(0, noise_level, size=n_samples)
    data = pd.concat([pd.Series(y, name="y"), X_df], axis=1)

    # Сохранение исходных данных
    data.to_csv(output_file, index=False)
    print(f"Исходные данные сохранены в файл '{output_file}'.")

    # Генерация новых данных для предсказания
    X_new = np.random.uniform(0, 10, size=(n_new_samples, n_factors))
    X_new_df = pd.DataFrame(X_new, columns=[f"X{i+1}" for i in range(n_factors)])
    X_new_df.to_csv(new_data_file, index=False)
    print(f"Новые данные для предсказания сохранены в файл '{new_data_file}'.")

    # Дополнительно: вывод коэффициентов для справки
    print("Использованные коэффициенты модели:")
    for i, coef in enumerate(coefficients, start=1):
        print(f"X{i}: {coef:.4f}")
    print(f"Уровень шума (стд. отклонение): {noise_level}")


def main():
    parser = argparse.ArgumentParser(
        description="Линейная Многофакторная Модель (ЛМФМ)"
    )
    parser.add_argument(
        "--generate", action="store_true", help="Сгенерировать синтетические данные."
    )
    parser.add_argument(
        "--n_samples",
        type=int,
        default=100,
        help="Количество наблюдений для исходных данных.",
    )
    parser.add_argument("--n_factors", type=int, default=3, help="Количество факторов.")
    parser.add_argument(
        "--coefficients", type=float, nargs="+", help="Коэффициенты для факторов."
    )
    parser.add_argument("--noise_level", type=float, default=1.0, help="Уровень шума.")
    parser.add_argument(
        "--output_file",
        type=str,
        default="data.csv",
        help="Имя файла для исходных данных.",
    )
    parser.add_argument(
        "--new_data_file",
        type=str,
        default="new_data.csv",
        help="Имя файла для новых данных.",
    )
    parser.add_argument(
        "--n_new_samples",
        type=int,
        default=10,
        help="Количество новых наблюдений для предсказания.",
    )
    parser.add_argument(
        "--random_state",
        type=int,
        default=None,
        help="Сид для генератора случайных чисел.",
    )

    args = parser.parse_args()

    if args.generate:
        generate_data(
            n_samples=args.n_samples,
            n_factors=args.n_factors,
            coefficients=args.coefficients,
            noise_level=args.noise_level,
            output_file=args.output_file,
            new_data_file=args.new_data_file,
            n_new_samples=args.n_new_samples,
            random_state=args.random_state,
        )
        return

Пример использования

```sh
python generate_data.py --generate --n_samples 200 --n_factors 5 --coefficients 2.5 1.5 3.0 4.5 2.0 --noise_level 2.0 --output_file data.csv --new_data_file new_data.csv --n_new_samples 20 --random_state 21

Исходные данные сохранены в файл 'data.csv'.
Новые данные для предсказания сохранены в файл 'new_data.csv'.
Использованные коэффициенты модели:
X1: 2.5000
X2: 1.5000
X3: 3.0000
X4: 4.5000
X5: 2.0000
Уровень шума (стд. отклонение): 2.0
```
